In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from scipy.io import arff
import pandas as pd

arff_data = arff.loadarff('/content/drive/MyDrive/NetworkTrafficNet/ScenarioB/TimeBasedFeatures-Dataset-15s.arff')
# arff_data = arff.loadarff('/content/drive/MyDrive/NetworkTrafficNet/ScenarioB/TimeBasedFeatures-Dataset-15s-AllinOne.arff')
# arff_data = arff.loadarff('/content/drive/MyDrive/NetworkTrafficNet/ScenarioB/TimeBasedFeatures-Dataset-30s.arff')
# arff_data = arff.loadarff('/content/drive/MyDrive/NetworkTrafficNet/ScenarioB/TimeBasedFeatures-Dataset-30s-AllinOne.arff')
# arff_data = arff.loadarff('/content/drive/MyDrive/NetworkTrafficNet/ScenarioB/TimeBasedFeatures-Dataset-60s.arff')
# arff_data = arff.loadarff('/content/drive/MyDrive/NetworkTrafficNet/ScenarioB/TimeBasedFeatures-Dataset-60s-AllinOne.arff')
# arff_data = arff.loadarff('/content/drive/MyDrive/NetworkTrafficNet/ScenarioB/TimeBasedFeatures-Dataset-120s.arff')
# arff_data = arff.loadarff('/content/drive/MyDrive/NetworkTrafficNet/ScenarioB/TimeBasedFeatures-Dataset-120s-AllinOne.arff')
data = pd.DataFrame(arff_data[0])
data.head()

,duration,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,flowPktsPerSecond,flowBytesPerSecond,min_flowiat,max_flowiat,mean_flowiat,std_flowiat,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,class1
0,9368711.0,16.0,4.0,1564818.0,1549373.0,190205.285714,203290.456522,389822.391917,370323.719754,10.353612,4802.688438,4.0,1523088.0,97590.739583,267600.198443,1871488.0,1.983656e+06,2195089.0,1.832197e+05,1234883.0,1420565.0,1523088.0,161096.539275,b'CHAT'
1,7340238.0,18.0,4.0,1567554.0,1527893.0,165686.977273,186914.846154,317267.548742,304370.651301,11.580006,4340.186245,4.0,1517774.0,87383.785714,221462.862028,1491627.0,3.572433e+06,5653239.0,2.942704e+06,1131498.0,1324636.0,1517774.0,273138.379008,b'CHAT'
2,4644225.0,29.0,15.0,1270547.0,1079974.0,165865.178571,195302.130435,329473.126261,300492.588227,11.412022,4161.512416,15.0,1079974.0,89312.019231,217475.425246,1758922.0,1.758922e+06,1758922.0,0.000000e+00,1079974.0,1079974.0,1079974.0,0.000000,b'CHAT'
3,4978735.0,19.0,8.0,2492050.0,2457286.0,239543.250000,276596.388889,612435.304238,628339.573544,8.034169,3918.666087,8.0,2441990.0,127659.871795,436959.716436,1710925.0,2.382905e+06,3054885.0,9.503232e+05,1346073.0,1894031.5,2441990.0,774930.342317,b'CHAT'
4,11838189.0,19.0,10.0,3094089.0,3093543.0,243766.500000,295954.725000,599721.781709,625632.703972,7.602514,2802.540152,10.0,3042717.0,133013.359551,436129.639296,1747431.0,2.400446e+06,3240696.0,6.232744e+05,1394455.0,1983227.0,3042717.0,725987.829075,b'CHAT'
